In [1]:
interval=10 #seconds

In [2]:
import datetime
import sys
import random
import gzip
import json
from tqdm import tqdm
import pickle

In [2]:
data = []
with open('place_tiles.csv', 'r') as f:
    f.readline()
    count=0
    #Useless but I'm adding it for fun
    nextcount=random.randrange(3000,20000)
    prevcount=0
    for line in f:
        count+=1
        if (prevcount+nextcount<=count):
            sys.stdout.write(f"\r{count} lines processed")
            sys.stdout.flush()
            prevcount=count
            nextcount=random.randrange(3000,20000)
        line = [x.rstrip() for x in line.split(",")]
        try:
            line[0]=datetime.datetime.strptime(line[0], '%Y-%m-%d %H:%M:%S.%f %Z')
        except:
            line[0]=datetime.datetime.strptime(line[0], '%Y-%m-%d %H:%M:%S %Z')
        if (line[2]=='' or line[3]=='' or line[4]==''):
            continue
        line[0]=int(line[0].timestamp()*1000)
        line[2]=int(line[2])
        line[3]=int(line[3])
        line[4]=int(line[4])
        data.append(line)
    sys.stdout.write(f"\r{count} lines processed")
    sys.stdout.flush()

16567567 lines processed

In [14]:
# import pickle
# with open("data.pickle", "wb") as f:
#     pickle.dump(data, f)


In [3]:
with open("data.pickle", "rb") as f:
    data = pickle.load(f)

In [4]:
print("Sorting...",end="")
data.sort()
print("Done")

Sorting...Done


In [5]:
with gzip.open('segments/1490172525.json.gz', 'rb') as f:
    file_content=f.read().decode('utf8')

In [6]:
hi = json.loads(file_content)

In [8]:
hi['events']

[{'id': 0,
  'timems': 1490133335548,
  'user': 'Kg7sd4cyQflqAT6w75IKeQ==',
  'x': 500,
  'y': 504,
  'color': 2},
 {'id': 1,
  'timems': 1490133347515,
  'user': 'Kg7sd4cyQflqAT6w75IKeQ==',
  'x': 508,
  'y': 503,
  'color': 2},
 {'id': 2,
  'timems': 1490133359619,
  'user': 'Kg7sd4cyQflqAT6w75IKeQ==',
  'x': 509,
  'y': 504,
  'color': 2},
 {'id': 3,
  'timems': 1490133373689,
  'user': 'Kg7sd4cyQflqAT6w75IKeQ==',
  'x': 509,
  'y': 503,
  'color': 2},
 {'id': 4,
  'timems': 1490133385251,
  'user': 'Kg7sd4cyQflqAT6w75IKeQ==',
  'x': 509,
  'y': 502,
  'color': 2},
 {'id': 5,
  'timems': 1490139263757,
  'user': 'esKDe9vhDcHuGP5RfwFwQw==',
  'x': 501,
  'y': 502,
  'color': 4},
 {'id': 6,
  'timems': 1490139275693,
  'user': 'esKDe9vhDcHuGP5RfwFwQw==',
  'x': 501,
  'y': 517,
  'color': 5},
 {'id': 7,
  'timems': 1490139291423,
  'user': 'esKDe9vhDcHuGP5RfwFwQw==',
  'x': 496,
  'y': 502,
  'color': 12},
 {'id': 8,
  'timems': 1490139307891,
  'user': 'esKDe9vhDcHuGP5RfwFwQw==',
  '

In [6]:
data[:100]

[[1490133335548, 'Kg7sd4cyQflqAT6w75IKeQ==', 500, 504, 2],
 [1490133347515, 'Kg7sd4cyQflqAT6w75IKeQ==', 508, 503, 2],
 [1490133359619, 'Kg7sd4cyQflqAT6w75IKeQ==', 509, 504, 2],
 [1490133373689, 'Kg7sd4cyQflqAT6w75IKeQ==', 509, 503, 2],
 [1490133385251, 'Kg7sd4cyQflqAT6w75IKeQ==', 509, 502, 2],
 [1490139263757, 'esKDe9vhDcHuGP5RfwFwQw==', 501, 502, 4],
 [1490139275693, 'esKDe9vhDcHuGP5RfwFwQw==', 501, 517, 5],
 [1490139291423, 'esKDe9vhDcHuGP5RfwFwQw==', 496, 502, 12],
 [1490139307891, 'esKDe9vhDcHuGP5RfwFwQw==', 499, 498, 2],
 [1490139322680, 'esKDe9vhDcHuGP5RfwFwQw==', 511, 514, 12],
 [1490144589019, 'esKDe9vhDcHuGP5RfwFwQw==', 149, 491, 2],
 [1490144603884, 'esKDe9vhDcHuGP5RfwFwQw==', 0, 0, 3],
 [1490172525177, 'Kg7sd4cyQflqAT6w75IKeQ==', 492, 403, 2],
 [1490172526888, 'Kg7sd4cyQflqAT6w75IKeQ==', 492, 404, 2],
 [1490172529287, 'Kg7sd4cyQflqAT6w75IKeQ==', 492, 405, 2],
 [1490172530922, 'Kg7sd4cyQflqAT6w75IKeQ==', 492, 406, 2],
 [1490172533003, 'Kg7sd4cyQflqAT6w75IKeQ==', 493, 406, 2],

In [7]:
colors = ["#FFFFFF",
    "#E4E4E4",
    "#888888",
    "#222222",
    "#FFA7D1",
    "#E50000",
    "#E59500",
    "#A06A42",
    "#E5D900",
    "#94E044",
    "#02BE01",
    "#00E5F0",
    "#0083C7",
    "#0000EA",
    "#E04AFF",
    "#820080"
]

In [8]:
def package(field, arr, time):
    data = dict()
    data['field']=field
    temp=[]
    for i in range(len(arr)):
        cur=arr[i]
        temp.append({'id':i, 'timems': cur[0], 'user': cur[1], 'x': cur[2], 'y': cur[3], 'color': cur[4]})
    data['events']=temp
    data['sizex']=len(field)
    data['sizey']=len(field[0])
    data['basetime']=time
    file = gzip.open(f"segments/{time}.json.gz", "wb")
    file.write(json.dumps(data).encode('utf8'))
    file.close()

In [9]:
hardcodedstart=1490172525

In [10]:
field,log=[[[0,'Unchanged']]*1000]*1000,[]
chunk=data[0][0]//1000
if hardcodedstart!=0: chunk=hardcodedstart
nextchunk=chunk+interval
chunkcount=0
cont = True
for i in tqdm(range(0, len(data))):
    currow=data[i]
    ts,user,x,y,color=currow[0]//1000,currow[1],currow[2],currow[3],currow[4]
    while (currow[0]>nextchunk*1000):
        if chunkcount>50:
            cont=False
            break
        chunkcount+=1
        package(field, log, chunk)
        log=[]
        chunk=nextchunk
        nextchunk+=interval
    if not cont: break
    field[x][y]=[color, user]
    log.append(currow)

  0%|          | 179/16567564 [00:25<661:41:33,  6.95it/s]


In [6]:
import json

In [9]:
test = dict()
test['hi']=[[0,1],[2,3]]
json.dumps(test)

'{"hi": [[0, 1], [2, 3]]}'